<a href="https://colab.research.google.com/github/denisabrantesredis/denisd-redis-learning-sessions/blob/main/JSON/JSON.ipynb" target="_newt">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<div style="display:flex;width=100%;">
<img src="https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120" alt="Redis" width="90"/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</div>

# Redis Learning Session - JSON Data Type

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/banner.png?raw=true" alt="Redis Data Types"/>

In this notebook, we will explore the JSON data type supported by Redis.

## Before We Start

<a href="https://redis.io/try-free/" target="_new">
<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/ClientSideCaching/_assets/images/callout_rediscloud.png?raw=true" alt="Callout - Create free Redis Cloud Account"/>
</a>

<b>Create a new free Redis Cloud account: <a href="https://redis.io/try-free/" target="_new">Click Here</a>

## Installing the Pre-Reqs

In [ ]:
!pip install -q redis
!pip install -q ipython-autotime

In [ ]:
%load_ext autotime

## Installing Redis Locally
If you are not using Redis Cloud as a database, uncomment and run the code below to install Redis locally. Then set your connection to 127.0.0.1

In [ ]:
# %%sh
# curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
# echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
# sudo apt-get update  > /dev/null 2>&1
# sudo apt-get install redis-stack-server  > /dev/null 2>&1
# redis-stack-server --daemonize yes

## Copying and Unzipping Lab Files

In [ ]:
import os

In [ ]:
if not os.path.exists("./files"):
  !mkdir files
  !wget https://github.com/denisabrantesredis/denisd-redis-learning-sessions/raw/refs/heads/main/JSON/_assets/files/prod_data.json
  !mv prod_data.json ./files

## Connecting to Redis

In [ ]:
import redis
from google.colab import userdata

#### Setup the Connection String

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_secrets.png?raw=true" alt="Callout - Use Google Colab secrets instead"/>

In [ ]:
try:
  REDIS_HOST = userdata.get('REDIS_HOST')
except:
  REDIS_HOST="127.0.0.1"

try:
  REDIS_PORT = userdata.get('REDIS_PORT')
except:
  REDIS_PORT=6379

try:
  REDIS_PASSWORD = userdata.get('REDIS_PASSWORD')
except:
  REDIS_PASSWORD=""

REDIS_URL = f"redis://default:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

#### Testing the Connection to Redis

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_connection.png?raw=true" alt="Callout - Make sure connection works"/>

In [ ]:
r = redis.from_url(
    REDIS_URL,
    protocol=3,
    decode_responses=True)

if r.ping():
    print("Connection successful!")
else:
    print("Connection issue!")

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

Make sure you can connect to the database using Redis Insight. The database should be empty for now.

&nbsp;

&nbsp;

## JSON as String Data

First, let's test reading and writing JSON as Strings.

This is our customer's JSON document:

In [ ]:
import json

In [ ]:
customer_data = {
    "id": "1234",
    "fist_name": "John",
    "last_name": "Doe",
    "email": "john.doe@gmail.com",
    "address": {
        "street": "123 Main Street",
        "city": "Chicago",
        "state": "IL",
        "zipcode": "60606"
    },
    "phone_numbers": [
        {"type": "home", "number": "555-0001", "status": "active"},
        {"type": "mobile", "number": "555-0002", "status": "active"},
        {"type": "fax", "number": "555-0003", "status": "deactivated"}
    ]
}

We'll need to serialize this variable as a String in order to store it in Redis:

In [ ]:
customer_str = json.dumps(customer_data)
print(customer_str)

r.set("customer:1234", customer_str)

Let's read the key value from Redis:

In [ ]:
r.get("customer:1234")

Let's say we want to update the customer's mobile number. There is no easy way to do this, if the data is stored as a String key. First, we need to retrieve the data, then parse it from String to JSON:

In [ ]:
customer_str = r.get("customer:1234")

customer_data = json.loads(customer_str)
print(customer_data)

Next, we update the property and verify:

In [ ]:
customer_data["phone_numbers"][1]["number"] = "555-1111"
print(customer_data)

Then we need to parse it back to String and save it to Redis:

In [ ]:
customer_str = json.dumps(customer_data)

r.set("customer:1234", customer_str)

Finally, we verify the new value:

In [ ]:
r.get("customer:1234")

## JSON as JSON Data

Let's try the same operation using the native JSON data type:

In [ ]:
customer_data = {
    "id": "1234",
    "fist_name": "John",
    "last_name": "Doe",
    "email": "john.doe@gmail.com",
    "address": {
        "street": "123 Main Street",
        "city": "Chicago",
        "state": "IL",
        "zipcode": "60606"
    },
    "phone_numbers": [
        {"type": "home", "number": "555-0001", "status": "active"},
        {"type": "mobile", "number": "555-0002", "status": "active"},
        {"type": "fax", "number": "555-0003", "status": "deactivated"}
    ]
}

Save to Redis:

In [ ]:
r.json().set("customer:0001234", "$", customer_data)

&nbsp;

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

Open Redis Insight and confirm that the Stream key was generated. You should see both the String key and the JSON key, since we've used different key names.

&nbsp;

&nbsp;

Now we update this customer's mobile number:

In [ ]:
r.json().set("customer:0001234", "$.phone_numbers[1].number", "555-1111")

You can use Redis Insight to confirm the new value, or run this:

In [ ]:
r.json().get("customer:0001234", "$.phone_numbers")

## Load JSON Product data

First, we need to import the Products dataset into Redis:

In [ ]:
import os
import json
import numpy as np

from redis.commands.search.field import TagField
from redis.commands.search.field import TextField
from redis.commands.search.field import NumericField
from redis.commands.search.query import NumericFilter, Query
from redis.commands.search.index_definition import IndexDefinition, IndexType

In [ ]:
with open('./files/prod_data.json', 'r') as file:
    prod_ds = json.load(file)

In [ ]:
prod_ds[0]

In [ ]:
pipe = r.pipeline(transaction=False)

In [ ]:
for prod in prod_ds:
    keyname = f"product:{prod['id']}"
    pipe.json().set(keyname, "$", prod)

results = pipe.execute()
len(results)

&nbsp;

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

Use Insight to make sure the new JSON keys are there.

&nbsp;
&nbsp;

Next, let's create a search index for the product keys:

In [ ]:
schema = (
    NumericField("$.id", as_name="id", sortable=True),
    NumericField("$.price", as_name="price"),
    NumericField("$.discountedPrice", as_name="discountedPrice"),
    TextField("$.articleNumber", as_name="articleNumber"),
    TextField("$.productDisplayName", as_name="productDisplayName"),
    TextField("$.productDescription", as_name="productDescription", index_missing=True, index_empty=True),
    TextField("$.variantName", as_name="variantName"),
    NumericField("$.catalogAddDate", as_name="catalogAddDate"),
    TagField("$.brandName", as_name="brandName"),
    TagField("$.ageGroup", as_name="ageGroup"),
    TagField("$.gender", as_name="gender"),
    TagField("$.baseColour", as_name="baseColour"),
    TagField("$.fashionType", as_name="fashionType"),
    TagField("$.season", as_name="season"),
    TagField("$.year", as_name="year"),
    NumericField("$.rating", as_name="rating"),
    TagField("$.displayCategories", as_name="displayCategories"),
    TagField("$.masterCategory", as_name="masterCategory"),
    TagField("$.subCategory", as_name="subCategory"),
    TextField("$.articleType", as_name="articleType"),
    NumericField("$.discount_pct", as_name="discount_pct"),
    NumericField("$.inventoryCount", as_name="inventoryCount", index_missing=True),
    TextField("$.company.name", as_name="vendorName"),
    NumericField("$.company.score", as_name="vendorScore", sortable=True),
    TagField("$.colors[*]", as_name="colors"),
    TagField("$.sizes[*]", as_name="sizes")
)
try:
    r.ft("idx:product").dropindex()
except:
    print("--> JSONProd index doesn't exist; creating it")
try:
  definition = IndexDefinition(prefix=["product:"], index_type=IndexType.JSON)
  result = r.ft("idx:product").create_index(fields=schema, definition=definition)
except Exception as ex:
    result = f"FAILED to create index: {ex}"

Make sure the index is populated:

In [ ]:
info = r.ft('idx:product').info()
print(f" Percent Indexed: {int(info['percent_indexed'])*100}")
print(f" Total Documents: {info['num_docs']}")

Now we can test some more advanced commands to manipulate JSON data

&nbsp;

## Advanced JSON Commands

### Retrieving Multiple Attributes

Let's says that we need to display (on our application's UI) the product's description, brand and price.

We can get all the data we need with a single command:

In [ ]:
prod_id = "product:10110"

r.json().get(prod_id, "$.productDisplayName", "$.brandName", "$.price")

We can also access our nested documents and arrays. For instance, besides the 3 attributes from before, I also need the vendor score and the first color available for this product:

In [ ]:
r.json().get(prod_id, "$.productDisplayName", "$.brandName", "$.price", "$.company.score", "$.colors[0]")

The recursive descent operator `..` can retrieve a field from multiple sections of a JSON document.

Here, we want to get the value of the `score` attribute, no matter where it is in the document:

In [ ]:
r.json().get(prod_id, "$..score")

Keep in mind that if this document had many `score` attributes, they would all be retrieved.

For instance, let's retrieve all phone numbers from the customer key we created before:

In [ ]:
r.json().get("customer:0001234", "$..number")

&nbsp;

### Array Operations

Adding new elements to an array is very simple. For instance, let's add a new color to this product:

In [ ]:
prod_id = "product:10110"

r.json().arrappend(prod_id, "$.colors", "pink")

The `ARRAPPEND` command, however, will always add the new value at the end of the list. In some cases, we may want to control the position of this new value in the list.

The `ARRINSERT` command can be used instead, to give us this control:

In [ ]:
r.json().arrinsert(prod_id, "$.colors", 0, "red")

To be clear, `ARRINSERT` will not override the original value; it will 'push' the other values in the list.

We can also insert multiple values at once, setting the index for the first element:

In [ ]:
r.json().arrinsert(prod_id, "$.colors", 1, "magenta", "grey", "tan", "yellow")

&nbsp;

<img src="https://github.com/denisabrantesredis/denisd-redis-learning-sessions/blob/main/Streams/_assets/images/callout_insight.png?raw=true" alt="Callout - Check Redis Insight"/>

Use Insight to see the new values in the product key.

&nbsp;
&nbsp;

The `ARRLEN` command allows us to quickly see how many colors are available for the product:

In [ ]:
r.json().arrlen(prod_id, "$.colors")

The `ARRINDEX` command can be used to find the position of an item in an array:

In [ ]:
magenta_index = r.json().arrindex(prod_id, "$.colors", "magenta")[0]
magenta_index

It's possible to remove elements from an array, using the `ARRPOP` command:

In [ ]:
r.json().arrpop(prod_id, "$.colors", magenta_index)

&nbsp;

### Other Operations

`STRLEN` returns the length of a string attribute of a JSON element. This can be used to check if an attribute is empty. For instance, does our product have a display name available:

In [ ]:
r.json().strlen(prod_id, "$.productDisplayName")

It's possible to increase or reduce the value of numerical attributes using the `NUMINCRBY` command.

Let's increase the score of our vendor, from 1 to 8:

In [ ]:
r.json().numincrby(prod_id, "$.company.score", 7)

Oops, went too far! Let's bring it down to 5:

In [ ]:
r.json().numincrby(prod_id, "$.company.score", -3)

We manipulate boolean values in JSON documents. First, let's set a new attribute that contains a Boolean value. We will retrieve the `discount_pct` attribute, and if it has a value of zero, we will set a `isDiscounted` attribute to False; otherwise, we will set it to True.

In [ ]:
discount_pct = r.json().get(prod_id, "$.discount_pct")[0]
print(discount_pct)

if discount_pct == 0:
    r.json().set(prod_id, "$.isDiscounted", False)
else:
    r.json().set(prod_id, "$.isDiscounted", True)

Let's say that a new discount was added to this product; we need to flip the value from False to True.

This can be done with the `TOGGLE` command.

In [ ]:
r.json().toggle(prod_id, "isDiscounted")

With `OBJLEN`, we can count the number of keys in a JSON document:

In [ ]:
r.json().objlen(prod_id)

It can also be used with documents nested within the root document:

In [ ]:
r.json().objlen(prod_id, "$.company")

The `OBJKEYS` command returns a list of keys for a JSON document:

In [ ]:
r.json().objkeys(prod_id)

As with the previous command, it can be used with nested documents:

In [ ]:
r.json().objkeys(prod_id, "$.company")

Finally, the `DEL` (or `FORGET`) command can be used to delete elements of a JSON document (or the entire key).

Let's remove the `sizes` list from our product key:

In [ ]:
r.json().delete(prod_id, "$.sizes")

Or we can set the path to root (`$`) and delete the entire key:

In [ ]:
r.json().delete(prod_id, "$")

&nbsp;
&nbsp;

## Search

### Filter

JSONPath supports filtering operations, which can quickly validate if the document contains a certain value.

For instance, check if this key has a price lower than 500 and a score greater than zero.

In [ ]:
prod_id = "product:10112"

r.json().get(prod_id, "$.[?($.price < 500 && $.company.score > 0)]")

Next, retrieve only the 'active' phone numbers for our customer 1234:

In [ ]:
r.json().get("customer:0001234", "$.phone_numbers[?(@.status == 'active')]")

And here is another way to retrieve just the phone numbers:

In [ ]:
r.json().get("customer:0001234", "$.phone_numbers[*].number")

We can also filter lists by value. For instance, retrieve all available colors for this product that start with the letter 'S':

In [ ]:
prod_id = "product:10184"

r.json().get(prod_id, "$.colors[?(@ =~ 'S.*')]")

Notice, however, that all those operations apply to individual keys. In order to apply those filters to multiple keys, we need to use Redis Search capabilities.

&nbsp;

### Redis Search

We have an [entire lab](https://tinyurl.com/redislssearch) on Redis Search, so we will keep this section short.

For instance, show me the 10 most expensive products available in the 'Navy' color:

In [ ]:
query = Query('@colors:{Navy}'
                ).sort_by('price', asc=False
                ).return_fields('id', 'productDisplayName', 'articleType', 'price', 'colors'
                ).paging(0,10)

results = r.ft("idx:product").search(query)['results']
for result in results:
  print(result['extra_attributes'])

Or list products that have 'Nike Women Shoe' in the description:

In [ ]:
query = Query(f'@productDisplayName:{"Nike Women Shoe"}'
                ).sort_by('id', asc=True
                ).return_fields('productDisplayName', 'price', 'id'
                ).paging(0,10)

results = r.ft("idx:product").search(query)['results']
for result in results:
  print(result['extra_attributes'])

&nbsp;
&nbsp;
&nbsp;
# Congrats, this is the end of the lab!!